In [0]:
%tensorflow_version 1.x

In [0]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
import pickle
with open('intents.json') as file:
  data= json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and AMEX', 'We accept most major credit cards']}, {'tag': 'opentoday', 'patterns': ['Are you open today?', 'When do you op

In [0]:
words=[]
labels=[]
doc_x=[]
doc_y=[]

In [0]:
for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds= nltk.word_tokenize(pattern)
    words.extend(wrds)
    doc_x.append(wrds)
    doc_y.append(intent['tag'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])


In [0]:
words= [stemmer.stem(w.lower()) for w in words if w != '?']
words= sorted(list(set(words)))
labels= sorted(labels)

In [0]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(labels,open('labels.pkl','wb'))

In [0]:
training=[]
output=[]
output_empty=[0 for _ in range(len(labels))]
for x, doc in enumerate(doc_x):
  wrds= [stemmer.stem(w.lower()) for w in doc]
  bag=[]
  for word in words:
    if word in wrds:
      bag.append(1)
    else:
      bag.append(0)

    output_row = output_empty[:]
    output_row[labels.index(doc_y[x])] = 1


    training.append(bag)
    output.append(output_row)


    

  

In [0]:
training= np.array(training)
output= np.array(output)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
model= Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
try:
  model.load('chatbot_model.h5')
except:
  hist = model.fit(np.array(training), np.array(output), epochs=200, batch_size=5, verbose=1)
  model.save('chatbot_model.h5')
  print("model created")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200





640/640 [==============================] - 1s 1ms/step - loss: 1.7552 - acc: 0.2328
Epoch 2/200
640/640 [==============================] - 0s 269us/step - loss: 1.6079 - acc: 0.3813
Epoch 3/200
640/640 [==============================] - 0s 267us/step - loss: 1.4328 - acc: 0.5250
Epoch 4/200
640/640 [==============================] - 0s 253us/step - loss: 1.2528 - acc: 0.6094
Epoch 5/200
640/640 [==============================] - 0s 239us/step - loss: 1.0595 - acc: 0.6766
Epoch 6/200
640/640 [==============================] - 0s 237us/step - loss: 0.9219 - acc: 0.7297
Epoch 7/200
640/640 [==============================] - 0s 236us/step - loss: 0.7549 - acc: 0.8125
Epoch 8/200
640/640 [==============================] - 0s 233us/step - loss: 0.6356 - acc: 0.8

In [0]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()